# Import the necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# Load and prepare the data

In [ ]:
# Load the dataset
df = pd.read_csv('Dataset/dataset.csv')

# Drop irrelevant columns
df.drop(columns=['index'], inplace=True)

# Separate features and target variable
X = df.drop(columns=['Result'])  # Features
y = df['Result']  # Target

# Handle missing values
X.fillna(X.median(), inplace=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and Train the XGBoost model

In [ ]:
# Initialize XGBoost model
model = XGBClassifier(random_state=42, n_jobs=-1, use_label_encoder=False, eval_metric='logloss')

# Define parameter grid for GridSearchCV
param_grid = {
    'max_depth': [3, 5, 7],  # Shallower trees to reduce complexity
    'n_estimators': [100, 200],  # Fewer estimators to reduce computation
    'learning_rate': [0.01, 0.1],  # Lower learning rate for better generalization
    'subsample': [0.8, 1.0],  # Subsample ratio to prevent overfitting
    'colsample_bytree': [0.8, 1.0]  # Feature subsampling to reduce overfitting
}

# Perform Grid Search with 5-fold Cross-Validation
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)



In [ ]:
# Get Best Parameters and Best Accuracy
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters Found:", best_params)
print(f"Best Cross-Validation Accuracy: {best_score:.4f}")

# Train the model with best parameters
best_model = XGBClassifier(**best_params, random_state=42, n_jobs=-1, use_label_encoder=False, eval_metric='logloss')
best_model.fit(X_train, y_train)

# Evaluate the model

In [ ]:
# Make Predictions
y_pred = best_model.predict(X_test)

# Evaluate Model Performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Final Test Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

# Save the model

In [ ]:
import joblib
joblib.dump(best_model, 'Trained_Models/Final_XGBoost_model1.pkl')
print("XGBoost Model Trained and Saved")